# Plot RMS changes 

This notebook will be plotting the differences in RMS due to a synthetic magma body at depth

In [1]:
from pathlib import Path
from mtpy import MTData, MTCollection
from mtpy.modeling.plots.plot_modem_rms import PlotRMS
from mtpy.modeling.modem.data import Data
import numpy as np

In [2]:
h5_path = Path(r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\cl_magma_tests.h5")

In [3]:
with MTCollection() as mc:
    mc.open_collection(h5_path)
    md = mc.to_mt_data()

25:05:09T15:13:23 | INFO | line:769 |mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\cl_magma_tests.h5


In [4]:
def calculate_rms(data_df, model_df):
    for col in ["z_xx", "z_xy", "z_yx", "z_yy", "t_zx", "t_zy"]:
        with np.errstate(divide="ignore", invalid="ignore"):
            model_df[f"rms_{col.replace('_', '')}"] = np.abs(
                data_df[col] - model_df[col]
            ) / (np.real(data_df[f"{col}_model_error"]) * np.sqrt(2))
    return model_df

In [5]:
def calculate_rms_change(model_01_df, model_02_df):
    """ 
    Assumes 1 is the original, 2 is the new and returns model 2 as 2 - 1

    In that sense if the value is positive then 2 is larger rms and if
    the value is negative then 2 got smaller.
    """
    for col in ["z_xx", "z_xy", "z_yx", "z_yy", "t_zx", "t_zy"]:
        key = f"rms_{col.replace('_', '')}"
        with np.errstate(divide="ignore", invalid="ignore"):
            model_02_df[key] = (model_02_df[key] - model_01_df[key]) / model_02_df[key]

    return model_02_df

In [6]:
data_obj = Data()
data_df = data_obj.read_data_file(Path(
    r"c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\inv_05_topo\cl_modem_data_z03_t02_tec_07.dat"
))

In [7]:
sorted(md.survey_ids)

['17km_10ohmm',
 '17km_20ohmm',
 '17km_30ohmm',
 '17km_3ohmm',
 '60km_10ohmm',
 '60km_30ohmm',
 '60km_3ohmm',
 'data',
 'gpc_10ohmm',
 'gpc_20ohmm',
 'gpc_30ohmm',
 'gpc_3ohmm',
 'inv_z03_t02_c02_040',
 'inv_z03_t02_c02x2_028',
 'li17km_10ohmm',
 'li17km_30ohmm',
 'li17km_3ohmm']

In [8]:
md_dict = {}
for survey in sorted(md.survey_ids):
    md_dict[survey] = md.get_survey(survey).to_mt_dataframe()

In [9]:
md_dict["data"] = data_df

In [10]:
base_model_01 = calculate_rms(md_dict["data"].dataframe, md_dict["inv_z03_t02_c02_040"].dataframe)
base_model_02 = calculate_rms(md_dict["data"].dataframe, md_dict["inv_z03_t02_c02x2_028"].dataframe)

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.cm import ScalarMappable
import matplotlib.ticker as ticker

# Create a segmented colormap with white band in the middle
def create_segmented_bwr_cmap():
    # Define colors for segments (blue to white to red)
    blues = plt.cm.Blues(np.linspace(0.3, 1, 5))  # 5 shades of blue
    whites = np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])  # 2 white segments for -0.1 to 0.1
    reds = plt.cm.Reds(np.linspace(0, 0.7, 5))  # 5 shades of red
    
    # Combine the colors
    colors_array = np.vstack([blues[::-1], whites, reds])
    
    # Create nodes for the segments (ensure we have white at -0.1 to 0.1)
    nodes = np.concatenate([
        np.linspace(-1, -0.1, 6),  # 5 blue segments
        np.linspace(-0.1, 0.1, 2),  # white segment
        np.linspace(0.1, 1, 6)      # 5 red segments
    ])
    
    # Create the colormap
    # cmap = colors.LinearSegmentedColormap.from_list('SegmentedBWR', list(zip(nodes, colors_array)))
    cmap = colors.LinearSegmentedColormap.from_list('SegmentedBWR', list(colors_array))
    return cmap

In [12]:
rms_cmap = create_segmented_bwr_cmap()

In [13]:
model_df = calculate_rms(md_dict["data"].dataframe, md_dict["li17km_3ohmm"].dataframe)
delta_model_df = calculate_rms_change(base_model_01, model_df)

In [14]:
for key in md_dict.keys():
    if key in ["data", 'inv_z03_t02_c02_040', 'inv_z03_t02_c02x2_028']:
        continue
    try:
        model_df = calculate_rms(md_dict["data"].dataframe, md_dict[key].dataframe)
    except ValueError:
        print(f"Skipping {key}")
        continue
    for base_df, base_id in zip([base_model_01, base_model_02], [40, 28]):
        delta_model_df = calculate_rms_change(base_df, model_df)

        dp = PlotRMS(delta_model_df)
        dp.rms_min = -.1
        dp.rms_max = .08
        dp.rms_step = .02
        dp.fig_num = 4
        dp.fig_size = [20, 10]
        #dp.rms_cmap = "mt_rd2wh2bl"
        dp.rms_cmap = rms_cmap
        dp.plot_station = False
        dp.box_size = 20
        dp.dx = .0175

        dp.plot()
        dp.save_plot(h5_path.parent.joinpath(f"{key}_vs_{base_id}_delta_rms.png"), fig_dpi=300)

Skipping 17km_10ohmm
25:05:09T15:20:38 | WARNING | line:198 |mtpy.modeling.plots.plot_modem_rms | _plot_rms_map | Could not add base map because HTTPSConnectionPool(host='basemap.nationalmap.gov', port=443): Max retries exceeded with url: /arcgis/rest/services/USGSTopo/MapServer/tile/10/390/161 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))
25:05:09T15:20:41 | INFO | line:125 |mtpy.imaging.mtplot_tools.base | save_plot | Saved figure to: c:\Users\jpeacock\OneDrive - DOI\ClearLake\modem_inv\magma_tests\17km_20ohmm_vs_40_delta_rms.png
25:05:09T15:20:44 | WARNING | line:198 |mtpy.modeling.plots.plot_modem_rms | _plot_rms_map | Could not add base map because HTTPSConnectionPool(host='basemap.nationalmap.gov', port=443): Max retries exceeded with url: /arcgis/rest/services/USGSTopo/MapServer/tile/10/390/161 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIF